[source](https://www.mathworks.com/help/optim/ug/sudoku-puzzles-problem-based.html)

In [1]:
import numpy as np
import scipy as sp
import gurobipy as gp
from gurobipy import GRB
import highspy

from utils.timer import Timer

from numba import njit
import numba


In [2]:
#The sudocu from the problem
empty_sudoku = np.array(
   [[0, 0, 1, 0, 0, 0, 0, 0, 2],
    [0, 0, 0, 3, 0, 4, 5, 0, 0],
    [6, 0, 0, 7, 0, 0, 1, 0, 0],
    [0, 4, 0, 5, 0, 0, 0, 0, 0],
    [0, 2, 0, 0, 0, 0, 0, 8, 0],
    [0, 0, 0, 0, 0, 6, 0, 9, 0],
    [0, 0, 5, 0, 0, 9, 0, 0, 4],
    [0, 0, 8, 2, 0, 1, 0, 0, 0],
    [3, 0, 0, 0, 0, 0, 7, 0, 0]]
)

#a sample sudocu from the internet
sample_solution = np.array(
    [[7, 1, 3, 5, 2, 4, 6, 9, 8],
     [5, 2, 9, 6, 1, 8, 3, 4, 7],
     [6, 4, 8, 7, 3, 9,	2, 5, 1],
     [1, 5, 2, 9, 4, 7, 8, 3, 6],
     [8, 3, 6, 1, 5, 2, 9, 7, 4],
     [4, 9, 7, 3, 8, 6, 5, 1, 2],
     [3, 8, 5, 4, 6, 1, 7, 2, 9],
     [9, 6,	1, 2, 7, 5, 4, 8, 3],
     [2, 7, 4, 8, 9, 3, 1, 6, 5]]
)

#the same sample sudoku with some elements deleted
empty_sample_solution = np.array(
    [[0, 0, 0, 5, 2, 4, 6, 9, 8],
     [5, 2, 0, 6, 1, 8, 3, 4, 7],
     [6, 4, 8, 7, 3, 9,	2, 5, 1],
     [1, 5, 2, 9, 4, 7, 8, 3, 6],
     [8, 3, 6, 1, 5, 2, 9, 7, 4],
     [4, 9, 7, 3, 8, 6, 5, 1, 2],
     [3, 8, 5, 4, 6, 1, 7, 2, 9],
     [9, 6,	1, 2, 7, 5, 4, 8, 3],
     [2, 7, 4, 8, 9, 3, 1, 6, 5]]
)

#A very empty sudoku
very_empty_clues = np.zeros((9,9), dtype = int)
very_empty_clues[0,1] = 3
very_empty_clues[0,2] = 2
very_empty_clues[0,3] = 1



In [3]:
#you can also just turn the matrix into a sparse one in index format, that might be faster
def sudoku_array_to_clues(clue_array):
    clues_inds_rows, clues_inds_cols = np.where(clue_array>0)
    clues_vals = clue_array[clues_inds_rows, clues_inds_cols]
    return clues_inds_rows, clues_inds_cols, clues_vals

@njit
def sudoku_array_to_clues_njit(clue_array):
    clues_inds_rows, clues_inds_cols = np.where(clue_array>0)
    clues_vals = np.empty_like(clues_inds_rows) 
    for i, (row_ind, col_ind) in enumerate(zip(clues_inds_rows, clues_inds_cols)):
        clues_vals[i] = clue_array[row_ind, col_ind]
    return clues_inds_rows, clues_inds_cols, clues_vals

def array_to_vector(sudoku_array):
    #turn a sudoku values array into a binary vector
    flat_values = sudoku_array.flatten()
    binary_values = np.zeros((flat_values.shape[0], 9), dtype = int)
    indices = np.arange(flat_values.shape[0], dtype = int)
    binary_values[indices, flat_values-1] = 1
    return binary_values.flatten()

def vector_to_array(binary_solution):
    #turn a binary vector into a soduku values array
    binary_values = binary_solution.reshape((81,9)) #magig numbers are sudoky dims and number of values possible
    _, flat_values = np.where(binary_values)
    sudoku_array = (flat_values+1).reshape(9,9)
    return sudoku_array

bin_sample_solution = array_to_vector(sample_solution)
vector_to_array(bin_sample_solution)
    

array([[7, 1, 3, 5, 2, 4, 6, 9, 8],
       [5, 2, 9, 6, 1, 8, 3, 4, 7],
       [6, 4, 8, 7, 3, 9, 2, 5, 1],
       [1, 5, 2, 9, 4, 7, 8, 3, 6],
       [8, 3, 6, 1, 5, 2, 9, 7, 4],
       [4, 9, 7, 3, 8, 6, 5, 1, 2],
       [3, 8, 5, 4, 6, 1, 7, 2, 9],
       [9, 6, 1, 2, 7, 5, 4, 8, 3],
       [2, 7, 4, 8, 9, 3, 1, 6, 5]])

In [4]:
#these are operatiors that should apply to a 729 long solution vector x
#x 
NCOLS = 9
NROWS = 9
NVALS = 9

def generate_one_value_constr_sparse(format = "csr", dtype = int):
    #[1,1,1,1,1,1,1,1,1, 0,0,0,0,0,....]
    #[0,0,0,0,0,0,0,0,0, 1,1,1,1,1,....
    #...
    return sp.sparse.kron(
        sp.sparse.eye(NROWS*NCOLS, dtype = dtype, format=format),
        np.ones((1,NVALS), dtype = dtype),
        format=format
    )

def generate_one_value_constr_dense(dtype = int):
    #[1,1,1,1,1,1,1,1,1, 0,0,0,0,0,....]
    #[0,0,0,0,0,0,0,0,0, 1,1,1,1,1,....
    #...
    return np.kron(
        np.eye(NROWS*NCOLS, dtype = dtype),
        np.ones((1,NVALS), dtype = dtype)
    )

@njit
def generate_one_value_constr_njit():
    #[1,1,1,1,1,1,1,1,1, 0,0,0,0,0,....]
    #[0,0,0,0,0,0,0,0,0, 1,1,1,1,1,....
    #...
    return np.kron(
        np.eye(NROWS*NCOLS, dtype=numba.boolean),
        np.ones((1,NVALS), dtype=numba.boolean)
    )


def generate_row_uniq_constr_sparse(format = "csr", dtype = int):
    #[1,..,0, 1,..,0, ... 1,... 0,  0,1,..,0, 0, 
    return sp.sparse.kron(
        sp.sparse.eye(NROWS, dtype = dtype, format=format),
        sp.sparse.kron(np.ones((1,NCOLS), dtype = dtype), 
                       sp.sparse.eye(NVALS, dtype = dtype),
                       format=format),
        format=format
    )

def generate_row_uniq_constr_dense(dtype = int):
    #[1,..,0, 1,..,0, ... 1,... 0,  0,1,..,0, 0, 
    return np.kron(
        np.eye(NROWS, dtype = dtype),
        np.kron(np.ones((1,NCOLS), dtype = dtype), 
                       np.eye(NVALS, dtype =dtype))
    )

@njit
def generate_row_uniq_constr_njit():
    #[1,..,0, 1,..,0, ... 1,... 0,  0,1,..,0, 0, 
    return np.kron(
        np.eye(NROWS, dtype = numba.boolean),
        np.kron(np.ones((1,NCOLS), dtype = numba.boolean), 
                       np.eye(NVALS, dtype = numba.boolean))
    )


def generate_col_uniq_constr_sparse(format = "csr", dtype = int):
    return sp.sparse.kron(
        np.ones((1,NROWS), dtype = dtype),
        sp.sparse.eye(NCOLS*NVALS, dtype = dtype, format=format),
        format=format
    )

def generate_col_uniq_constr_dense(dtype = int):
    return np.kron(
        np.ones((1,NROWS), dtype = dtype),
        np.eye(NCOLS*NVALS, dtype = dtype)
    )

@njit
def generate_col_uniq_constr_njit():
    return np.kron(
        np.ones((1,NROWS), dtype = numba.boolean),
        np.eye(NCOLS*NVALS, dtype = numba.boolean)
    )
    

def generate_block_uniq_constr_sparse(format = "csr", dtype = int):
    return sp.sparse.kron(
        sp.sparse.eye(3, dtype = dtype, format=format),
        sp.sparse.kron(
            np.ones((1,3), dtype = dtype), 
            sp.sparse.kron(
                sp.sparse.eye(3, dtype = dtype, format=format),
                sp.sparse.kron( 
                    np.ones((1,3), dtype = dtype),
                    sp.sparse.eye(9, dtype = dtype, format=format)
                ),
                format=format
            ),
            format=format   
        ),
        format=format
    )

def generate_block_uniq_constr_dense(dtype = int):
    return np.kron(
        np.eye(3, dtype = dtype),
        np.kron(
            np.ones((1,3), dtype = dtype), 
            np.kron(
                np.eye(3, dtype = dtype),
                np.kron( 
                    np.ones((1,3), dtype = dtype),
                    np.eye(9, dtype = dtype)
                ),
            ),
        ),
    )

@njit
def generate_block_uniq_constr_njit():
    return np.kron(
        np.eye(3, dtype = numba.boolean),
        np.kron(
            np.ones((1,3), dtype = numba.boolean), 
            np.kron(
                np.eye(3, dtype = numba.boolean),
                np.kron( 
                    np.ones((1,3), dtype = numba.boolean),
                    np.eye(9, dtype = numba.boolean)
                ),
            ),
        ),
    )

def generate_clues_contr_sparse(clues_inds_rows, clues_inds_cols, clues_vals,
                         format = "csr", dtype = int):
    #x[i,j,m] = 1
    op_shape = (clues_vals.shape[0], NROWS*NCOLS*NVALS)

    data = np.ones((clues_vals.shape[0]), dtype=dtype)
    row_inds = np.arange(clues_vals.shape[0])
    col_inds = NVALS*NCOLS*clues_inds_rows + NVALS*clues_inds_cols + clues_vals-1
    op_sparse = sp.sparse.csc_matrix( (data, (row_inds, col_inds)), shape = op_shape)
    return op_sparse

def generate_clues_contr_dense(clues_inds_rows, clues_inds_cols, clues_vals, dtype = int):
    #x[i,j,m] = 1
    op_shape = (clues_vals.shape[0], NROWS*NCOLS*NVALS)
    op_dense = np.zeros(op_shape, dtype = dtype)

    row_inds = np.arange(clues_vals.shape[0])
    col_inds = NVALS*NCOLS*clues_inds_rows + NVALS*clues_inds_cols + clues_vals-1
    op_dense[row_inds, col_inds] = 1
    return op_dense

@njit
def generate_clues_contr_njit(clues_inds_rows, clues_inds_cols, clues_vals):
    #x[i,j,m] = 1
    op_shape = (clues_vals.shape[0], NROWS*NCOLS*NVALS)
    op_dense = np.zeros(op_shape, dtype = numba.boolean)
    
    col_inds = NVALS*NCOLS*clues_inds_rows + NVALS*clues_inds_cols + clues_vals-1
    for i, col_ind in enumerate(col_inds):
        op_dense[i, col_ind] = True
    return op_dense



In [5]:
def setup_ILP_sparse(clue_array, format = "csr", dtype = int):
    clues_inds_rows, clues_inds_cols, clues_vals = sudoku_array_to_clues(clue_array)
    
    A_one_val = generate_one_value_constr_sparse(format = format, dtype = dtype)
    A_row = generate_row_uniq_constr_sparse(format = format, dtype = dtype)
    A_col = generate_col_uniq_constr_sparse(format = format, dtype = dtype)
    A_block = generate_block_uniq_constr_sparse(format = format, dtype = dtype)
    A_clues = generate_clues_contr_sparse(clues_inds_rows, clues_inds_cols, clues_vals, format = format, dtype = dtype)

    A_full = sp.sparse.vstack((A_one_val, A_row, A_col, A_block, A_clues), format = format)
    c = np.linspace(0,1,A_full.shape[1]) #np.zeros(solution_size, dtype = np.float64)
    return c, A_full

def setup_ILP_dense(clue_array, dtype = int):
    clues_inds_rows, clues_inds_cols, clues_vals = sudoku_array_to_clues(clue_array)
    
    A_one_val = generate_one_value_constr_dense(dtype = dtype)
    A_row = generate_row_uniq_constr_dense(dtype = dtype)
    A_col = generate_col_uniq_constr_dense(dtype = dtype)
    A_block = generate_block_uniq_constr_dense(dtype = dtype)
    A_clues = generate_clues_contr_dense(clues_inds_rows, clues_inds_cols, clues_vals, dtype = dtype)

    A_full = np.vstack((A_one_val, A_row, A_col, A_block, A_clues))
    c = np.linspace(0,1,A_full.shape[1]) #np.zeros(solution_size, dtype = np.float64)
    return c, A_full

def setup_ILP_sparse_end(clue_array, format = "csr", dtype = int):
    clues_inds_rows, clues_inds_cols, clues_vals = sudoku_array_to_clues(clue_array)
    
    A_one_val = generate_one_value_constr_dense(dtype = dtype)
    A_row = generate_row_uniq_constr_dense(dtype = dtype)
    A_col = generate_col_uniq_constr_dense(dtype = dtype)
    A_block = generate_block_uniq_constr_dense(dtype = dtype)
    A_clues = generate_clues_contr_dense(clues_inds_rows, clues_inds_cols, clues_vals, dtype = dtype)

    A_full = sp.sparse.vstack((A_one_val, A_row, A_col, A_block, A_clues), format = format)
    c = np.linspace(0,1,A_full.shape[1]) #np.zeros(solution_size, dtype = np.float64)
    return c, A_full

@njit
def setup_ILP_njit(clue_array):
    clues_inds_rows, clues_inds_cols, clues_vals = sudoku_array_to_clues_njit(clue_array)
    
    A_one_val = generate_one_value_constr_njit()
    A_row = generate_row_uniq_constr_njit()
    A_col = generate_col_uniq_constr_njit()
    A_block = generate_block_uniq_constr_njit()
    A_clues = generate_clues_contr_njit(clues_inds_rows, clues_inds_cols, clues_vals)

    A_full = np.vstack((A_one_val, A_row, A_col, A_block, A_clues))
    c = np.linspace(0,1,A_full.shape[1]) #np.zeros(solution_size, dtype = np.float64)
    return c, A_full
    

In [6]:
with Timer("njitted"):
    setup_ILP_njit(empty_sudoku)

with Timer("normal"):
    setup_ILP_dense(empty_sudoku)

print(Timer.timers)

{'njitted': [3.1373010200913996], 'normal': [0.002184425015002489]}


In [48]:


def solve_sudoku_scipy(c, A_full):
    solution_size = A_full.shape[1]
    integrality = np.ones(solution_size, dtype = int)
    bounds = sp.optimize.Bounds(lb=0, ub=1) #somthing with 0 and 
    constraints = sp.optimize.LinearConstraint(A_full, lb=1, ub=1) #somthing with A
    res = sp.optimize.milp(c, integrality = integrality, bounds = bounds, constraints = constraints)
    return res

env = gp.Env(empty=True)
env.setParam('OutputFlag', 0)
env.start()
        
def solve_sudoku_gurobi(c, A_full):
    # Create a new model
    m = gp.Model("matrix1", env = env) 
    # Create variables
    solution_size = A_full.shape[1]
    x = m.addMVar(shape=solution_size, vtype=GRB.BINARY, name="x")
    # Build rhs vector
    rhs = np.ones(A_full.shape[0], dtype = int)
    # Add constraints
    m.addConstr(A_full @ x == rhs, name="constraints")
    # Set objective
    m.setObjective(c @ x, GRB.MINIMIZE)
    # Optimize model
    m.optimize()
    return x.X
    

def solve_sudoku_highspy(c, A_full):

    rows, cols = A_full.shape
    # Highs h
    h = highspy.Highs()

    # Define a HighsLp instance
    lp = highspy.HighsLp()
    
    lp.num_col_ = cols
    lp.num_row_ = rows
    lp.col_cost_ = c
    lp.col_lower_ = np.zeros(c.shape, dtype = int)
    lp.col_upper_ = np.ones(c.shape, dtype = int)
    lp.row_lower_ = np.ones((rows), dtype = int)
    lp.row_upper_ = np.ones((rows), dtype = int)
    lp.integrality_ = [highspy._core.HighsVarType.kInteger]*cols # np.ones(c.shape, dtype = int).tolist()
    #lp.integrality_ = 1#np.ones(c.shape, dtype = int)#highspy.HighsVarType.kInteger
    
    # In a HighsLp instsance, the number of nonzeros is given by a fictitious final start
    lp.a_matrix_.format_ = highspy.MatrixFormat.kColwise
    lp.a_matrix_.start_ = A_full.indptr
    lp.a_matrix_.index_ = A_full.indices
    lp.a_matrix_.value_ = A_full.data
    h.passModel(lp)

    # Get and set options
    options = h.getOptions()
    options.log_to_console = False
    h.passOptions(options)

    h.run()
    
    solution = h.getSolution()
    return np.array(solution.col_value)

from pyscipopt import Model
model = Model()

def solve_sudoku_pisciopt(c, A_full):
    num_constr, num_vars = A_full.shape 
    x = model.addVar("x", num_vars, vtype="binary")
    #model.setObjective(c @ x)
    #model.addCons(A_full@x == 0)
    #model.addCons(2*x - y*y >= 0)
    
    return model

        





In [49]:
with Timer("Solve Highs"):
    c, A_full_njit = setup_ILP_njit(empty_sudoku)
    A_full_njit_sparse = sp.sparse.csc_matrix(A_full_njit)
    #model = solve_sudoku_pisciopt(c, A_full_njit)
    solution = solve_sudoku_highspy(c, A_full_njit_sparse)

print(vector_to_array(solution))
print(Timer.timers)

Running HiGHS 1.7.1 (git hash: 0c240d8): Copyright (c) 2024 HiGHS under MIT licence terms
[[7 3 1 8 6 5 9 4 2]
 [8 9 2 3 1 4 5 6 7]
 [6 5 4 7 9 2 1 3 8]
 [9 4 6 5 8 3 2 7 1]
 [1 2 3 9 4 7 6 8 5]
 [5 8 7 1 2 6 4 9 3]
 [2 7 5 6 3 9 8 1 4]
 [4 6 8 2 7 1 3 5 9]
 [3 1 9 4 5 8 7 2 6]]
{'njitted': [3.1373010200913996], 'normal': [0.002184425015002489], 'Solve Highs': [0.01014044089242816, 0.01246186112985015, 0.012220255099236965, 0.012605963973328471, 0.01210187398828566, 0.01242921408265829, 0.005345142912119627, 0.004742315039038658, 0.004716824973002076, 0.00547668500803411, 0.005147468065842986, 0.011789262993261218, 0.013099837116897106, 0.013716191984713078, 0.01254982384853065, 0.01327479793690145, 0.012615921907126904, 0.012198018841445446, 0.014960202854126692, 0.013174992986023426, 0.011703741969540715, 0.013462958857417107]}


In [21]:
highspy._core.HighsStatus

highspy._core.HighsStatus

In [9]:
raise Error

NameError: name 'Error' is not defined

In [ ]:

# %load_ext line_profiler
    
# %lprun -f setup_ILP_dense c, A_full_dense = setup_ILP_dense(empty_sudoku, dtype = bool)

# %lprun -f solve_sudoku_scipy res_scipy = solve_sudoku_scipy(c, A_full_dense)
# %lprun -f setup_ILP_sparse c, A_full_dense = setup_ILP_sparse(empty_sudoku, dtype = bool)



In [ ]:
#fastest option
with Timer("Solve sudoku"):
    c, A_full_njit = setup_ILP_njit(empty_sudoku)
    A_sparse_njit =  sp.sparse.csr_matrix(A_full_njit)
    res_scipy = solve_sudoku_scipy(c, A_sparse_njit)
    solve_sudoku_scipy(c, A_sparse_njit)
    

print(Timer.timers)
    
    

In [ ]:
raise Error

In [53]:
repeats = 4
for i in range(repeats):
    with Timer("1. Setup njit dense"):
        c, A_full_njit_dense = setup_ILP_njit(empty_sudoku)
    with Timer("2. Scipy solver njit dense"):
        res_scipy = solve_sudoku_scipy(c, A_full_njit_dense)
    with Timer("3. Gurobi solver njit dense"):
        res_gurobi = solve_sudoku_gurobi(c, A_full_njit_dense )
    del c
    del A_full_njit_dense 

    with Timer("1. Setup njit sparse"):
        c, A_full_njit_dense = setup_ILP_njit(empty_sudoku)
        A_full_njit_sparse = sp.sparse.csc_matrix(A_full_njit_dense)
    with Timer("2. Scipy solver njit sparse"):
        res_scipy = solve_sudoku_scipy(c, A_full_njit_sparse )
    with Timer("3. Gurobi solver njit sparse"):
        res_gurobi = solve_sudoku_gurobi(c, A_full_njit_sparse )
    with Timer("4. Highspy solver njit sparse"):
        res_highspy = solve_sudoku_highspy(c, A_full_njit_sparse )
    del c
    del A_full_njit_sparse
        
    with Timer("1. Setup sparse_dense"):
        c, A_full_sparse = setup_ILP_sparse_end(empty_sudoku, format = "csc", dtype = bool)
    with Timer("2. Scipy solver sparse_dense"):
        res_scipy = solve_sudoku_scipy(c, A_full_sparse)
    with Timer("3. Gurobi solver sparse_dense"):
        res_gurobi = solve_sudoku_gurobi(c, A_full_sparse)
    del c
    del A_full_sparse
    
    with Timer("1. Setup sparse"):
        c, A_full_sparse = setup_ILP_sparse(empty_sudoku, format = "csc", dtype = bool)
    with Timer("2. Scipy solver sparse"):
        res_scipy = solve_sudoku_scipy(c, A_full_sparse)
    with Timer("3. Gurobi solver sparse"):
        res_gurobi = solve_sudoku_gurobi(c, A_full_sparse)
    del c
    del A_full_sparse
    
    with Timer("1. Setup dense"):
        c, A_full_dense = setup_ILP_dense(empty_sudoku, dtype = bool)
    with Timer("2. Scipy solver dense"):
        res_scipy = solve_sudoku_scipy(c, A_full_dense)
    with Timer("3. Gurobi solver dense"):
        res_gurobi = solve_sudoku_gurobi(c, A_full_dense)
    del c
    del A_full_dense

np.set_printoptions(precision=4)
keys = np.sort(list(Timer.timers.keys()))
for key in keys:
    print(key, "\t\t", np.array(Timer.timers[key]))

Running HiGHS 1.7.1 (git hash: 0c240d8): Copyright (c) 2024 HiGHS under MIT licence terms
Running HiGHS 1.7.1 (git hash: 0c240d8): Copyright (c) 2024 HiGHS under MIT licence terms
Running HiGHS 1.7.1 (git hash: 0c240d8): Copyright (c) 2024 HiGHS under MIT licence terms
Running HiGHS 1.7.1 (git hash: 0c240d8): Copyright (c) 2024 HiGHS under MIT licence terms
1. Setup dense 		 [0.0007 0.0007 0.0007 0.0007 0.0007 0.0007 0.0007 0.0007 0.0007 0.0007
 0.0007 0.0007]
1. Setup njit dense 		 [0.0008 0.0006 0.0006 0.0007 0.0008 0.0007 0.0006 0.0006 0.0006 0.0008
 0.0007 0.0006 0.0006]
1. Setup njit sparse 		 [0.002  0.002  0.0019 0.002  0.002  0.002  0.002  0.002  0.0021 0.002
 0.002  0.002 ]
1. Setup sparse 		 [0.0038 0.0035 0.0035 0.0034 0.0036 0.0035 0.0035 0.0035 0.0037 0.0035
 0.0035 0.0035]
1. Setup sparse_dense 		 [0.0025 0.0024 0.0024 0.0024 0.0032 0.0024 0.0025 0.0025 0.0029 0.0024
 0.003  0.0026]
2. Scipy solver dense 		 [0.0104 0.0093 0.0093 0.0093 0.0095 0.0106 0.0093 0.0093 0.0096 0

In [ ]:
raise Error

In [ ]:
repeats = 4
for i in range(repeats):
    
    
    
    
    
print(vector_to_array(res_scipy.x))
print(vector_to_array(res_gurobi))

for key, value in Timer.timers.items():
    print(key, "\t\t", value)



In [ ]:
A_full

In [ ]:
raise Error